In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing import image
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.models import Sequential, Model
from keras.regularizers import l2
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D,Input,concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Main Folder Path
folder_path = "/kaggle/input/kermany2018/OCT2017 "

# Sub Folder Paths
train_dir = f"{folder_path}/train"
val_dir = f"{folder_path}/val"
test_dir = f"{folder_path}/test"

In [ ]:
os.listdir(folder_path)

In [ ]:
print(f"Train Directory: {os.listdir(train_dir)}")
print(f"Validation Directory: {os.listdir(test_dir)}")
print(f"Test Directory: {os.listdir(val_dir)}")

In [ ]:
normal_train_dir = os.path.join(train_dir, "NORMAL")
normal_train_files = os.listdir(normal_train_dir)[:30]

normal_train_files

## Baseline Model 1 — Custom CNN (grayscale 256×256)
We’ll verify a few images, build grayscale generators, define a compact CNN,
train with strong regularization, and evaluate on the test split.


In [ ]:
import random, os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

random.seed(42)

# Pick a random NORMAL image (change class to inspect others)
one_path = os.path.join(train_dir, "NORMAL", random.choice(os.listdir(os.path.join(train_dir, "NORMAL"))))
img = Image.open(one_path).convert("L")     # force grayscale
arr = np.array(img)
print("Path:", one_path)
print("PIL mode:", img.mode, " Shape:", arr.shape)

plt.imshow(arr, cmap="gray")
plt.axis("off")
plt.title("Sample (grayscale)")
plt.show()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 32
IMG_SIZE   = (256, 256)     # matches your earlier setup

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode="nearest",
)

eval_gen = ImageDataGenerator(rescale=1./255)

train_image_gen = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
)

val_image_gen = eval_gen.flow_from_directory(
    val_dir,                        # use "val" split for validation
    target_size=IMG_SIZE,
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
)

test_image_gen = eval_gen.flow_from_directory(
    test_dir,                       # held-out test
    target_size=IMG_SIZE,
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
)

num_classes = train_image_gen.num_classes
class_indices = train_image_gen.class_indices
print("Class indices:", class_indices)


In [ ]:
import os, numpy as np

cls2idx = train_image_gen.class_indices
counts = {c: len(os.listdir(os.path.join(train_dir, c))) for c in cls2idx}
total = sum(counts.values())
class_weights = {cls2idx[c]: total/(len(cls2idx)*counts[c]) for c in cls2idx}
print("Counts:", counts)
print("Class weights:", class_weights)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers

image_shape = (256, 256, 1)
L2 = regularizers.l2(1e-4)

model = models.Sequential([
    layers.Input(shape=image_shape),

    layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Dropout(0.10),

    layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Dropout(0.15),

    layers.Conv2D(128, 3, padding="same", activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Dropout(0.20),

    layers.Conv2D(128, 3, padding="same", activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Dropout(0.20),

    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu", kernel_initializer="he_normal", kernel_regularizer=L2),
    layers.Dropout(0.30),
    layers.Dense(num_classes, activation="softmax"),
])

model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.Adam(1e-3),
    metrics=["accuracy"],
)

model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

ckpt_path = "cnn_baseline_best.keras"
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, verbose=1),
    ModelCheckpoint(ckpt_path, monitor="val_accuracy", save_best_only=True, verbose=1),
    CSVLogger("cnn_baseline_log.csv", append=False),
]

EPOCHS = 15
history = model.fit(
    train_image_gen,
    validation_data=val_image_gen,
    epochs=EPOCHS,
    class_weight=class_weights,    # remove this arg if you don’t want balancing
    callbacks=callbacks,
    verbose=1,
)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

hist_df = pd.DataFrame(history.history)
hist_df.index = np.arange(1, len(hist_df)+1)

plt.figure(figsize=(10,5))
plt.plot(hist_df["loss"], label="loss")
plt.plot(hist_df["val_loss"], label="val_loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Training vs Validation Loss"); plt.legend(); plt.show()

plt.figure(figsize=(10,5))
plt.plot(hist_df["accuracy"], label="acc")
plt.plot(hist_df["val_accuracy"], label="val_acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title("Training vs Validation Accuracy"); plt.legend(); plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

test_loss, test_acc = model.evaluate(test_image_gen, verbose=1)
print(f"TEST — loss: {test_loss:.4f}  acc: {test_acc:.4f}")

pred_prob = model.predict(test_image_gen, verbose=1)
y_pred = pred_prob.argmax(axis=1)
y_true = test_image_gen.classes
target_names = list(test_image_gen.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=target_names, digits=4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix — CNN Baseline (Test)")
plt.show()


In [ ]:
model.save("CNN_model1_final.keras")


## Model 2 — InceptionV3 (grayscale → RGB, 299×299)


In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing import image
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, Model
from keras.regularizers import l2
from tensorflow.keras.layers import (
    Activation, Dropout, Dense, Flatten, Conv2D, BatchNormalization,
    MaxPooling2D, GlobalAveragePooling2D, Input, concatenate, Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import warnings
warnings.filterwarnings("ignore")

print("✅ Imports successful — TensorFlow version:", tf.__version__)


In [ ]:
# Main Folder Path
folder_path = "/kaggle/input/kermany2018/OCT2017 "

# Sub Folder Paths
train_dir = f"{folder_path}/train"
val_dir = f"{folder_path}/val"
test_dir = f"{folder_path}/test"

In [ ]:
os.listdir(folder_path)


In [ ]:
print(f"Train Directory: {os.listdir(train_dir)}")
print(f"Validation Directory: {os.listdir(test_dir)}")
print(f"Test Directory: {os.listdir(val_dir)}")

In [ ]:
# === Cell: Generators (train with aug; eval deterministic) ===
BATCH_SIZE = 32
IMG_SIZE   = (299, 299)  # InceptionV3 native

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.08,
    height_shift_range=0.08,
    zoom_range=0.10,
    brightness_range=(0.9, 1.1),
    horizontal_flip=True,
)

eval_aug = ImageDataGenerator(rescale=1./255)

train_gen = train_aug.flow_from_directory(
    train_dir, target_size=IMG_SIZE, color_mode="grayscale",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=True, seed=42
)
val_gen = eval_aug.flow_from_directory(
    val_dir, target_size=IMG_SIZE, color_mode="grayscale",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)
test_gen = eval_aug.flow_from_directory(
    test_dir, target_size=IMG_SIZE, color_mode="grayscale",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)

num_classes = train_gen.num_classes
class_names = list(train_gen.class_indices.keys())
print("Classes:", class_names)


In [ ]:
# === Cell: Optional class weights (helps imbalance) ===
cls2idx = train_gen.class_indices
counts = {c: len(os.listdir(os.path.join(train_dir, c))) for c in cls2idx}
total = sum(counts.values())
class_weights = {cls2idx[c]: total/(num_classes*counts[c]) for c in counts}
class_weights


In [ ]:
# === Cell: Build InceptionV3 (grayscale -> RGB, [-1,1] input) ===
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, Model, optimizers

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 1), name="gray_input")
x = layers.Concatenate(name="gray_to_rgb")([inputs, inputs, inputs])   # (H,W,3)
x = layers.Rescaling(scale=2.0, offset=-1.0, name="to_minus1_plus1")(x)

base = InceptionV3(weights="imagenet", include_top=False, input_tensor=x)
base.trainable = False  # Stage-1: freeze backbone

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dropout(0.30)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

iv3 = Model(inputs, outputs, name="InceptionV3_OCT")
iv3.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.Adam(1e-4),
    metrics=['accuracy']
)
iv3.summary()


In [ ]:
# === Cell: Stage-1 training (frozen backbone) ===
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

ckpt_s1 = "iv3_stage1_best.keras"
callbacks_s1 = [
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, verbose=1),
    ModelCheckpoint(ckpt_s1, monitor="val_loss", save_best_only=True, verbose=1),
    CSVLogger("iv3_stage1_log.csv", append=False),
]

EPOCHS_1 = 10
hist_1 = iv3.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_1,
    class_weight=class_weights,   # remove if you don't want weighting
    callbacks=callbacks_s1,
    verbose=1,
)


In [ ]:
# === Cell: Curves for stage-1 ===
import pandas as pd, matplotlib.pyplot as plt
s1 = pd.DataFrame(hist_1.history); s1.index += 1
plt.figure(figsize=(10,5)); plt.plot(s1["loss"], label="loss"); plt.plot(s1["val_loss"], label="val_loss"); plt.legend(); plt.title("Stage-1 Loss"); plt.show()
plt.figure(figsize=(10,5)); plt.plot(s1["accuracy"], label="acc"); plt.plot(s1["val_accuracy"], label="val_acc"); plt.legend(); plt.title("Stage-1 Accuracy"); plt.show()


In [ ]:
# === Cell: Curves for stage-1 ===
import pandas as pd, matplotlib.pyplot as plt
s1 = pd.DataFrame(hist_1.history); s1.index += 1
plt.figure(figsize=(10,5)); plt.plot(s1["loss"], label="loss"); plt.plot(s1["val_loss"], label="val_loss"); plt.legend(); plt.title("Stage-1 Loss"); plt.show()
plt.figure(figsize=(10,5)); plt.plot(s1["accuracy"], label="acc"); plt.plot(s1["val_accuracy"], label="val_acc"); plt.legend(); plt.title("Stage-1 Accuracy"); plt.show()


In [ ]:
# === Cell: Stage-2 fine-tuning (unfreeze top layers, lower LR) ===
for layer in base.layers[-50:]:
    layer.trainable = True

iv3.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.Adam(1e-5),
    metrics=['accuracy']
)

ckpt_s2 = "iv3_stage2_best.keras"
callbacks_s2 = [
    EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, verbose=1),
    ModelCheckpoint(ckpt_s2, monitor="val_loss", save_best_only=True, verbose=1),
    CSVLogger("iv3_stage2_log.csv", append=False),
]

EPOCHS_2 = 12
hist_2 = iv3.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_2,
    class_weight=class_weights,
    callbacks=callbacks_s2,
    verbose=1,
)


In [ ]:
# === Cell: Evaluate on test + classification report + confusion matrix ===
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

test_loss, test_acc = iv3.evaluate(test_gen, verbose=1)
print(f"TEST — loss: {test_loss:.4f} | acc: {test_acc:.4f}")

pred_prob = iv3.predict(test_gen, verbose=1)
y_pred = pred_prob.argmax(axis=1)
y_true = test_gen.classes
target_names = list(test_gen.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=target_names, digits=4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix — InceptionV3"); plt.show()


In [ ]:
# === Cell: True train accuracy (non-aug) to compare fairly with val ===
train_eval_gen = eval_aug.flow_from_directory(
    train_dir, target_size=IMG_SIZE, color_mode="grayscale",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)
tr_loss, tr_acc = iv3.evaluate(train_eval_gen, verbose=1)
vl_loss, vl_acc = iv3.evaluate(val_gen, verbose=1)
print(f"Train(no-aug) acc: {tr_acc:.4f} | Val acc: {vl_acc:.4f}")
